In [9]:
import pandas as pd
import numpy as np

import pandas_profiling as pd_prf

import os

In [10]:
os.getcwd()

'C:\\Users\\IPBAM065\\Documents'

In [11]:
std_data=pd.read_excel('Bridge i2i.xlsx')

In [12]:
std_data.shape

(3400, 56)

In [13]:
# Imputing missing values in stdnt_test_entrance_comb column based on mean of stdnt_major

std_data['STDNT_TEST_ENTRANCE_COMB'] = std_data['STDNT_TEST_ENTRANCE_COMB'].fillna(std_data.groupby('STDNT_MAJOR')['STDNT_TEST_ENTRANCE_COMB'].transform('mean'))

# To identify absentees 

std_data['absentee_hrs_f'] = std_data['FIRST_TERM_ATTEMPT_HRS'] - std_data['FIRST_TERM_EARNED_HRS'] # For 1st semester absence
std_data['absentee_hrs_s'] = std_data['SECOND_TERM_ATTEMPT_HRS'] - std_data['SECOND_TERM_EARNED_HRS'] # For 2nd semester absence

# Parents eduction level - considering based on highest of parents education

std_data['prnts_edn_cd'] = std_data[['FATHER_HI_EDU_CD','MOTHER_HI_EDU_CD']].max(axis=1)

# Values mapping for students grade

std_data['grade_1_f'] = std_data.CORE_COURSE_GRADE_1_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_2_f'] = std_data.CORE_COURSE_GRADE_2_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_3_f'] = std_data.CORE_COURSE_GRADE_3_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_4_f'] = std_data.CORE_COURSE_GRADE_4_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_5_f'] = std_data.CORE_COURSE_GRADE_5_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_6_f'] = std_data.CORE_COURSE_GRADE_6_F.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})

std_data['grade_1_s'] = std_data.CORE_COURSE_GRADE_1_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_2_s'] = std_data.CORE_COURSE_GRADE_2_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_3_s'] = std_data.CORE_COURSE_GRADE_3_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_4_s'] = std_data.CORE_COURSE_GRADE_4_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_5_s'] = std_data.CORE_COURSE_GRADE_5_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})
std_data['grade_6_s'] = std_data.CORE_COURSE_GRADE_6_S.map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'INCOMP':-1,'NOT REP':-1})

# Calculating grades average

grds_cols = ['grade_1_f','grade_2_f','grade_3_f','grade_4_f','grade_5_f','grade_6_f','grade_1_s','grade_2_s','grade_3_s','grade_4_s','grade_5_s','grade_6_s']
std_data['grd_avg'] = std_data[grds_cols].sum(axis=1)/std_data[grds_cols].count(axis=1)

# Grade band

std_data.loc[(std_data['grd_avg'] > 0) & (std_data['grd_avg'] <= 3) ,'grd_grp'] = 'Good'
std_data.loc[(std_data['grd_avg'] > 3) & (std_data['grd_avg'] <= 5) ,'grd_grp'] = 'Average'
std_data.loc[(std_data['grd_avg'] <= 0) | (std_data['grd_avg'] > 5) ,'grd_grp'] = 'Poor'

# Students secured marks >= 1340 in STDNT_TEST_ENTRANCE_COMB

std_data.loc[std_data['STDNT_TEST_ENTRANCE_COMB'] >= 1340,'entrance_score_ge1340'] = 1
std_data.loc[std_data['STDNT_TEST_ENTRANCE_COMB'] < 1340,'entrance_score_ge1340'] = 0

# Financial assistance required or not

std_data.loc[std_data['UNMET_NEED'] > 0,'fin_asst_req'] = 1
std_data.loc[std_data['UNMET_NEED'] <= 0,'fin_asst_req'] = 0

# International status flag

std_data.loc[std_data['INTERNATIONAL_STS'] == 'Y','intl_flag'] = 1
std_data.loc[std_data['INTERNATIONAL_STS'] == 'N','intl_flag'] = 0

# To identify which course students are leaving more - By removing numbers

std_data['course1'] = std_data.CORE_COURSE_NAME_1_F.str[0:4]
std_data['course2'] = std_data.CORE_COURSE_NAME_1_S.str[0:4]


std_data.loc[std_data['course1'].isin(['ENGL','HIST','CHEM','MATH','ECON']),'top5_subj'] = 1
std_data.loc[~std_data['course1'].isin(['ENGL','HIST','CHEM','MATH','ECON']),'top5_subj'] = 0

std_data.loc[std_data['course2'].isin(['ENGL','HIST','CHEM','MATH','COMM']),'top5_subj'] = 1
std_data.loc[~std_data['course2'].isin(['ENGL','HIST','CHEM','MATH','COMM']),'top5_subj'] = 0

# Returned 2nd year flag update


std_data['ret_2nd_yr'] = std_data.RETURNED_2ND_YR.map({0:1,1:0})

In [14]:
# Age cross tab

age_crosstab = pd.crosstab(std_data['STDNT_AGE'],std_data['RETURNED_2ND_YR'])/std_data.shape[0]*100
age_index = age_crosstab.reset_index()
age_index['col']=age_index[0]/(age_index[0] + age_index[1])*100
age_index

# Age band creation

std_data.loc[std_data['STDNT_AGE'] <= 16,'age_grp'] = 'age_le16'
std_data.loc[std_data['STDNT_AGE'] == 17,'age_grp'] = 'age_eq17'
std_data.loc[std_data['STDNT_AGE'] == 18,'age_grp'] = 'age_eq18'
std_data.loc[std_data['STDNT_AGE'] == 19,'age_grp'] = 'age_eq19'
std_data.loc[std_data['STDNT_AGE'] >= 20,'age_grp'] = 'age_ge20'

std_data.age_grp.value_counts()


age_eq18    2860
age_eq17     309
age_eq19     190
age_ge20      33
age_le16       8
Name: age_grp, dtype: int64

In [15]:
# Gender cross tab

gend_crosstab = pd.crosstab(std_data['STDNT_GENDER'],std_data['RETURNED_2ND_YR'])/std_data.shape[0]*100
gend_index = gend_crosstab.reset_index()
gend_index['col']=gend_index[0]/(gend_index[0] + gend_index[1])*100
gend_index

# Gender Flag

std_data.loc[std_data['STDNT_GENDER'] == 'M','gend_flag'] = 1
std_data.loc[std_data['STDNT_GENDER'] == 'F','gend_flag'] = 0

In [16]:
# Student Background cross tab

bkgrnd_crosstab = pd.crosstab(std_data['STDNT_BACKGROUND'],std_data['RETURNED_2ND_YR'])/std_data.shape[0]*100
bkgrnd_index = bkgrnd_crosstab.reset_index()
bkgrnd_index['col']=bkgrnd_index[0]/(bkgrnd_index[0] + bkgrnd_index[1])*100
bkgrnd_index

std_data.loc[std_data['STDNT_BACKGROUND'] == 'BGD 1','std_bkgrnd_grp'] = 'BKGRND_1'
std_data.loc[std_data['STDNT_BACKGROUND'] == 'BGD 2','std_bkgrnd_grp'] = 'BKGRND_2'
std_data.loc[std_data['STDNT_BACKGROUND'] == 'BGD 5','std_bkgrnd_grp'] = 'BKGRND_5'
std_data.loc[std_data['STDNT_BACKGROUND'] == 'BGD 7','std_bkgrnd_grp'] = 'BKGRND_7'
std_data.loc[std_data['STDNT_BACKGROUND'].isin(['BGD 3','BGD 4','BGD 6','BGD 8']),'std_bkgrnd_grp'] = 'OTHERS'

std_data.std_bkgrnd_grp.value_counts()




BKGRND_1    2082
OTHERS      1185
BKGRND_2     106
BKGRND_7      19
BKGRND_5       8
Name: std_bkgrnd_grp, dtype: int64

In [17]:
std_data.DEGREE_GROUP_CD.value_counts()

B    3384
A      12
V       4
Name: DEGREE_GROUP_CD, dtype: int64

In [18]:
# Dummies creation for 3 categorical variables
incl_dummies = pd.get_dummies(data=std_data, columns=['grd_grp','age_grp','std_bkgrnd_grp'])


In [19]:
drop_cols = [ 'STDNT_AGE','STDNT_GENDER', 'STDNT_BACKGROUND', 'INTERNATIONAL_STS', 'STDNT_MAJOR', 'STDNT_MINOR',
            'STDNT_TEST_ENTRANCE1', 'STDNT_TEST_ENTRANCE2','STDNT_TEST_ENTRANCE1', 'STDNT_TEST_ENTRANCE2',
            'FIRST_TERM', 'CORE_COURSE_NAME_1_F',
           'CORE_COURSE_GRADE_1_F', 'CORE_COURSE_NAME_2_F', 'CORE_COURSE_GRADE_2_F', 'CORE_COURSE_NAME_3_F',
           'CORE_COURSE_GRADE_3_F', 'CORE_COURSE_NAME_4_F', 'CORE_COURSE_GRADE_4_F', 'CORE_COURSE_NAME_5_F',
           'CORE_COURSE_GRADE_5_F', 'CORE_COURSE_NAME_6_F', 'CORE_COURSE_GRADE_6_F', 'SECOND_TERM', 'CORE_COURSE_NAME_1_S',
           'CORE_COURSE_GRADE_1_S', 'CORE_COURSE_NAME_2_S', 'CORE_COURSE_GRADE_2_S', 'CORE_COURSE_NAME_3_S',
           'CORE_COURSE_GRADE_3_S', 'CORE_COURSE_NAME_4_S', 'CORE_COURSE_GRADE_4_S', 'CORE_COURSE_NAME_5_S',
           'CORE_COURSE_GRADE_5_S', 'CORE_COURSE_NAME_6_S', 'CORE_COURSE_GRADE_6_S', 'HOUSING_STS', 'RETURNED_2ND_YR',
           'HIGH_SCHL_NAME', 'FATHER_HI_EDU_CD', 'FATHER_HI_EDU_DESC', 'MOTHER_HI_EDU_CD',
           'MOTHER_HI_EDU_DESC', 'DEGREE_GROUP_DESC', 'GROSS_FIN_NEED', 'UNMET_NEED',
            'grade_1_f', 'grade_2_f', 'grade_3_f', 'grade_4_f', 'grade_5_f', 'grade_6_f', 'grade_1_s', 'grade_2_s', 'grade_3_s',
       'grade_4_s', 'grade_5_s', 'grade_6_s', 'grd_avg','course1','course2','STUDENT IDENTIFIER',
             'grd_grp_Average','std_bkgrnd_grp_OTHERS', 'DEGREE_GROUP_CD', 'IN_STATE_FLAG',
             'std_bkgrnd_grp_BKGRND_1','std_bkgrnd_grp_BKGRND_2','std_bkgrnd_grp_BKGRND_5','std_bkgrnd_grp_BKGRND_7',
             'age_grp_age_eq17','age_grp_age_eq18','age_grp_age_eq19','age_grp_age_le16'
             
            ]

In [20]:
std_data1 = incl_dummies.drop(drop_cols,axis=1)

std_data1.isnull().sum()

STDNT_TEST_ENTRANCE_COMB      0
DISTANCE_FROM_HOME           25
HIGH_SCHL_GPA                53
FIRST_TERM_ATTEMPT_HRS        0
FIRST_TERM_EARNED_HRS         0
SECOND_TERM_ATTEMPT_HRS     206
SECOND_TERM_EARNED_HRS      209
COST_OF_ATTEND                0
EST_FAM_CONTRIBUTION          0
absentee_hrs_f                0
absentee_hrs_s              209
prnts_edn_cd                407
entrance_score_ge1340         0
fin_asst_req                  0
intl_flag                     0
top5_subj                     0
ret_2nd_yr                    0
gend_flag                     0
grd_grp_Good                  0
grd_grp_Poor                  0
age_grp_age_ge20              0
dtype: int64

In [21]:
# Observed missing values in absentee_hrs_s prnts_edn_cd and impute them with 0

std_data1.fillna(0, inplace=True)

std_data1.isnull().sum()

STDNT_TEST_ENTRANCE_COMB    0
DISTANCE_FROM_HOME          0
HIGH_SCHL_GPA               0
FIRST_TERM_ATTEMPT_HRS      0
FIRST_TERM_EARNED_HRS       0
SECOND_TERM_ATTEMPT_HRS     0
SECOND_TERM_EARNED_HRS      0
COST_OF_ATTEND              0
EST_FAM_CONTRIBUTION        0
absentee_hrs_f              0
absentee_hrs_s              0
prnts_edn_cd                0
entrance_score_ge1340       0
fin_asst_req                0
intl_flag                   0
top5_subj                   0
ret_2nd_yr                  0
gend_flag                   0
grd_grp_Good                0
grd_grp_Poor                0
age_grp_age_ge20            0
dtype: int64

In [22]:
# profile report preparation for EDA

#prof_rep = pd_prf.ProfileReport(std_data1)
#prof_rep.to_file('./profile_report.html')

In [23]:
# Random forest model building
X = std_data1.drop(['ret_2nd_yr'], axis=1)
Y = std_data1['ret_2nd_yr']

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.30,random_state=200)

from sklearn.ensemble import RandomForestClassifier

rf_mdl = RandomForestClassifier(n_estimators=173,oob_score=True,n_jobs=-1,random_state=200)

rf_mdl.fit(X_train,Y_train)

rf_mdl.oob_score_

0.8327731092436975

In [24]:
rf_mdl

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=173,
                       n_jobs=-1, oob_score=True, random_state=200, verbose=0,
                       warm_start=False)

In [25]:
cols = X_train.columns
for feature  in zip(cols, rf_mdl.feature_importances_):
    print(feature)

('STDNT_TEST_ENTRANCE_COMB', 0.09300186814184833)
('DISTANCE_FROM_HOME', 0.06393985378314425)
('HIGH_SCHL_GPA', 0.12233065930682929)
('FIRST_TERM_ATTEMPT_HRS', 0.04974533837023983)
('FIRST_TERM_EARNED_HRS', 0.062420330022363746)
('SECOND_TERM_ATTEMPT_HRS', 0.14769593917165208)
('SECOND_TERM_EARNED_HRS', 0.13742693445436213)
('COST_OF_ATTEND', 0.060432522618558165)
('EST_FAM_CONTRIBUTION', 0.05407751854226362)
('absentee_hrs_f', 0.03621489997428415)
('absentee_hrs_s', 0.03568767927989496)
('prnts_edn_cd', 0.04025831434227101)
('entrance_score_ge1340', 0.0019574029914880024)
('fin_asst_req', 0.014020774659555146)
('intl_flag', 0.00314428454295554)
('top5_subj', 0.02145038653278539)
('gend_flag', 0.020420058815347922)
('grd_grp_Good', 0.025721636879830964)
('grd_grp_Poor', 0.006365514765819152)
('age_grp_age_ge20', 0.003688082804506233)


In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

reg = DecisionTreeClassifier()
tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300]}
reg_cv = GridSearchCV(reg,tree_para,cv=5)
reg_cv.fit(X_train,Y_train)
print(reg_cv.best_params_)

{'criterion': 'entropy', 'max_depth': 4}


In [27]:
dt = DecisionTreeClassifier(criterion = 'gini', max_depth=4)

In [28]:
from sklearn.metrics import confusion_matrix

dt.fit(X_train,Y_train)
y_pred  = dt.predict(X_test)
species = np.array(Y_test)
predictions = np.array(y_pred)
confusion_matrix(species,predictions)

array([[790,  11],
       [166,  53]], dtype=int64)

In [30]:
for estimator in range(10000,11000,100):
    clf=RandomForestClassifier(n_estimators=estimator,oob_score=True,n_jobs=-1,random_state=200)
    clf.fit(X_train,Y_train)
    oob=clf.oob_score_
    print('For n_estimators='+str(estimator))
    print('For OOB score is '+str(oob))
    print('****************************')
    

For n_estimators=100
For OOB score is 0.8352941176470589
****************************
For n_estimators=200
For OOB score is 0.8323529411764706
****************************
For n_estimators=300
For OOB score is 0.83109243697479
****************************
For n_estimators=400
For OOB score is 0.8285714285714286
****************************
For n_estimators=500
For OOB score is 0.8285714285714286
****************************
For n_estimators=600
For OOB score is 0.8294117647058824
****************************
For n_estimators=700
For OOB score is 0.8294117647058824
****************************
For n_estimators=800
For OOB score is 0.8306722689075631
****************************
For n_estimators=900
For OOB score is 0.8302521008403362
****************************
For n_estimators=1000
For OOB score is 0.8306722689075631
****************************
For n_estimators=1100
For OOB score is 0.8319327731092437
****************************
For n_estimators=1200
For OOB score is 0.8306722689075

KeyboardInterrupt: 

In [32]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [33]:
model = XGBClassifier()
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))


0.8127450980392157


In [34]:
model.feature_importances_

array([0.03926091, 0.03869862, 0.0401    , 0.03444132, 0.03591722,
       0.15674289, 0.06141247, 0.03700826, 0.03734697, 0.03568575,
       0.04413321, 0.03505912, 0.        , 0.05291399, 0.02953899,
       0.03449575, 0.04788096, 0.13988416, 0.0669358 , 0.03254364],
      dtype=float32)

In [35]:
model

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [41]:
xgclf = XGBClassifier()
parameters ={
    "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50],
    "gamma":[0,0.1,0.2,0.3,0.4,0.5,0.7,0.8,0.9,1.0]
    }

grid=GridSearchCV(xgclf,parameters,scoring="accuracy",cv=5)

In [42]:
grid.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     

In [40]:
y_pred=grid.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))

0.8225490196078431


In [43]:
grid.best_params_

{'gamma': 0, 'learning_rate': 0.05}

In [44]:
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))


0.8225490196078431


In [45]:
for feature  in zip(cols, model.feature_importances_):
    print(feature)

('STDNT_TEST_ENTRANCE_COMB', 0.031458613)
('DISTANCE_FROM_HOME', 0.035638466)
('HIGH_SCHL_GPA', 0.030573182)
('FIRST_TERM_ATTEMPT_HRS', 0.030051041)
('FIRST_TERM_EARNED_HRS', 0.02825461)
('SECOND_TERM_ATTEMPT_HRS', 0.2427785)
('SECOND_TERM_EARNED_HRS', 0.08219919)
('COST_OF_ATTEND', 0.028917806)
('EST_FAM_CONTRIBUTION', 0.030890454)
('absentee_hrs_f', 0.028970718)
('absentee_hrs_s', 0.035716925)
('prnts_edn_cd', 0.021714784)
('entrance_score_ge1340', 0.0)
('fin_asst_req', 0.046330184)
('intl_flag', 0.042636916)
('top5_subj', 0.021334548)
('gend_flag', 0.0319907)
('grd_grp_Good', 0.14222734)
('grd_grp_Poor', 0.045995045)
('age_grp_age_ge20', 0.042320922)


In [51]:
X_train=X_train.drop(['entrance_score_ge1340'], axis=1)
X_test=X_test.drop(['entrance_score_ge1340'], axis=1)

KeyError: "['entrance_score_ge1340'] not found in axis"

In [53]:
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))


0.8225490196078431


In [55]:
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

('STDNT_TEST_ENTRANCE_COMB', 0.031458613)
('DISTANCE_FROM_HOME', 0.035638466)
('HIGH_SCHL_GPA', 0.030573182)
('FIRST_TERM_ATTEMPT_HRS', 0.030051041)
('FIRST_TERM_EARNED_HRS', 0.02825461)
('SECOND_TERM_ATTEMPT_HRS', 0.2427785)
('SECOND_TERM_EARNED_HRS', 0.08219919)
('COST_OF_ATTEND', 0.028917806)
('EST_FAM_CONTRIBUTION', 0.030890454)
('absentee_hrs_f', 0.028970718)
('absentee_hrs_s', 0.035716925)
('prnts_edn_cd', 0.021714784)
('fin_asst_req', 0.046330184)
('intl_flag', 0.042636916)
('top5_subj', 0.021334548)
('gend_flag', 0.0319907)
('grd_grp_Good', 0.14222734)
('grd_grp_Poor', 0.045995045)
('age_grp_age_ge20', 0.042320922)


In [52]:
X_train.columns

Index(['STDNT_TEST_ENTRANCE_COMB', 'DISTANCE_FROM_HOME', 'HIGH_SCHL_GPA',
       'FIRST_TERM_ATTEMPT_HRS', 'FIRST_TERM_EARNED_HRS',
       'SECOND_TERM_ATTEMPT_HRS', 'SECOND_TERM_EARNED_HRS', 'COST_OF_ATTEND',
       'EST_FAM_CONTRIBUTION', 'absentee_hrs_f', 'absentee_hrs_s',
       'prnts_edn_cd', 'fin_asst_req', 'intl_flag', 'top5_subj', 'gend_flag',
       'grd_grp_Good', 'grd_grp_Poor', 'age_grp_age_ge20'],
      dtype='object')

In [56]:
X_train=X_train.drop(['prnts_edn_cd'], axis=1)
X_test=X_test.drop(['prnts_edn_cd'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8215686274509804
('STDNT_TEST_ENTRANCE_COMB', 0.030895667)
('DISTANCE_FROM_HOME', 0.033155464)
('HIGH_SCHL_GPA', 0.029872814)
('FIRST_TERM_ATTEMPT_HRS', 0.029025385)
('FIRST_TERM_EARNED_HRS', 0.03293979)
('SECOND_TERM_ATTEMPT_HRS', 0.24521056)
('SECOND_TERM_EARNED_HRS', 0.08497068)
('COST_OF_ATTEND', 0.028446736)
('EST_FAM_CONTRIBUTION', 0.032395583)
('absentee_hrs_f', 0.027915085)
('absentee_hrs_s', 0.037197404)
('fin_asst_req', 0.04767565)
('intl_flag', 0.046892338)
('top5_subj', 0.023720425)
('gend_flag', 0.033570297)
('grd_grp_Good', 0.14996478)
('grd_grp_Poor', 0.04418813)
('age_grp_age_ge20', 0.041963246)


In [57]:
X_train=X_train.drop(['age_grp_age_ge20'], axis=1)
X_test=X_test.drop(['age_grp_age_ge20'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8205882352941176
('STDNT_TEST_ENTRANCE_COMB', 0.03345042)
('DISTANCE_FROM_HOME', 0.03463281)
('HIGH_SCHL_GPA', 0.03158429)
('FIRST_TERM_ATTEMPT_HRS', 0.028213812)
('FIRST_TERM_EARNED_HRS', 0.03489321)
('SECOND_TERM_ATTEMPT_HRS', 0.25856093)
('SECOND_TERM_EARNED_HRS', 0.0883871)
('COST_OF_ATTEND', 0.029771179)
('EST_FAM_CONTRIBUTION', 0.03238875)
('absentee_hrs_f', 0.030469285)
('absentee_hrs_s', 0.038607452)
('fin_asst_req', 0.04533661)
('intl_flag', 0.04497674)
('top5_subj', 0.025020149)
('gend_flag', 0.035316568)
('grd_grp_Good', 0.15503027)
('grd_grp_Poor', 0.05336048)


In [58]:
X_train=X_train.drop(['COST_OF_ATTEND'], axis=1)
X_test=X_test.drop(['COST_OF_ATTEND'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8235294117647058
('STDNT_TEST_ENTRANCE_COMB', 0.035285436)
('DISTANCE_FROM_HOME', 0.038179692)
('HIGH_SCHL_GPA', 0.03715251)
('FIRST_TERM_ATTEMPT_HRS', 0.03346668)
('FIRST_TERM_EARNED_HRS', 0.036612418)
('SECOND_TERM_ATTEMPT_HRS', 0.2809446)
('SECOND_TERM_EARNED_HRS', 0.08984954)
('EST_FAM_CONTRIBUTION', 0.03209525)
('absentee_hrs_f', 0.031603023)
('absentee_hrs_s', 0.040684458)
('fin_asst_req', 0.031128932)
('intl_flag', 0.047752287)
('top5_subj', 0.022831712)
('gend_flag', 0.03708196)
('grd_grp_Good', 0.15411924)
('grd_grp_Poor', 0.051212236)


In [59]:
X_train=X_train.drop(['top5_subj'], axis=1)
X_test=X_test.drop(['top5_subj'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8225490196078431
('STDNT_TEST_ENTRANCE_COMB', 0.036785048)
('DISTANCE_FROM_HOME', 0.038552377)
('HIGH_SCHL_GPA', 0.034690812)
('FIRST_TERM_ATTEMPT_HRS', 0.030609578)
('FIRST_TERM_EARNED_HRS', 0.037123065)
('SECOND_TERM_ATTEMPT_HRS', 0.2771895)
('SECOND_TERM_EARNED_HRS', 0.09560184)
('EST_FAM_CONTRIBUTION', 0.032785594)
('absentee_hrs_f', 0.036265418)
('absentee_hrs_s', 0.043401632)
('fin_asst_req', 0.0360129)
('intl_flag', 0.04709686)
('gend_flag', 0.041651323)
('grd_grp_Good', 0.16108976)
('grd_grp_Poor', 0.05114428)


In [60]:
X_train=X_train.drop(['fin_asst_req'], axis=1)
X_test=X_test.drop(['fin_asst_req'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8225490196078431
('STDNT_TEST_ENTRANCE_COMB', 0.038856953)
('DISTANCE_FROM_HOME', 0.040423937)
('HIGH_SCHL_GPA', 0.037813634)
('FIRST_TERM_ATTEMPT_HRS', 0.03155206)
('FIRST_TERM_EARNED_HRS', 0.038595416)
('SECOND_TERM_ATTEMPT_HRS', 0.2730426)
('SECOND_TERM_EARNED_HRS', 0.09801451)
('EST_FAM_CONTRIBUTION', 0.034966655)
('absentee_hrs_f', 0.035159286)
('absentee_hrs_s', 0.04243262)
('intl_flag', 0.055705346)
('gend_flag', 0.038195945)
('grd_grp_Good', 0.17350748)
('grd_grp_Poor', 0.061733503)


In [61]:
X_train=X_train.drop(['gend_flag'], axis=1)
X_test=X_test.drop(['gend_flag'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8235294117647058
('STDNT_TEST_ENTRANCE_COMB', 0.04039194)
('DISTANCE_FROM_HOME', 0.036806524)
('HIGH_SCHL_GPA', 0.03721357)
('FIRST_TERM_ATTEMPT_HRS', 0.033709858)
('FIRST_TERM_EARNED_HRS', 0.034705896)
('SECOND_TERM_ATTEMPT_HRS', 0.33068427)
('SECOND_TERM_EARNED_HRS', 0.06793932)
('EST_FAM_CONTRIBUTION', 0.03455556)
('absentee_hrs_f', 0.028069144)
('absentee_hrs_s', 0.038779296)
('intl_flag', 0.047119156)
('grd_grp_Good', 0.2050092)
('grd_grp_Poor', 0.06501631)


In [62]:
for estimator in range(10000,11000,100):
    clf=RandomForestClassifier(n_estimators=estimator,oob_score=True,n_jobs=-1,random_state=200)
    clf.fit(X_train,Y_train)
    oob=clf.oob_score_
    print('For n_estimators='+str(estimator))
    print('For OOB score is '+str(oob))
    print('****************************')
    

For n_estimators=10000
For OOB score is 0.8281512605042017
****************************
For n_estimators=10100
For OOB score is 0.8281512605042017
****************************
For n_estimators=10200
For OOB score is 0.8281512605042017
****************************
For n_estimators=10300
For OOB score is 0.8281512605042017
****************************
For n_estimators=10400
For OOB score is 0.8281512605042017
****************************
For n_estimators=10500
For OOB score is 0.8281512605042017
****************************
For n_estimators=10600
For OOB score is 0.8281512605042017
****************************
For n_estimators=10700
For OOB score is 0.8277310924369747
****************************
For n_estimators=10800
For OOB score is 0.8277310924369747
****************************
For n_estimators=10900
For OOB score is 0.8277310924369747
****************************


In [65]:
X_train_new=X_train.drop(['absentee_hrs_f'], axis=1)
X_test_new=X_test.drop(['absentee_hrs_f'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train_new,Y_train)
y_pred=model.predict(X_test_new)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_train_new.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8205882352941176
('STDNT_TEST_ENTRANCE_COMB', 0.04172749)
('DISTANCE_FROM_HOME', 0.042000256)
('HIGH_SCHL_GPA', 0.039905056)
('FIRST_TERM_ATTEMPT_HRS', 0.035699066)
('FIRST_TERM_EARNED_HRS', 0.038488045)
('SECOND_TERM_ATTEMPT_HRS', 0.33010423)
('SECOND_TERM_EARNED_HRS', 0.077848)
('EST_FAM_CONTRIBUTION', 0.040034752)
('absentee_hrs_s', 0.044091243)
('intl_flag', 0.052837152)
('grd_grp_Good', 0.19071129)
('grd_grp_Poor', 0.06655339)


In [66]:
X_train_final=X_train.drop(['STDNT_TEST_ENTRANCE_COMB','DISTANCE_FROM_HOME','absentee_hrs_s'], axis=1)
X_test_final=X_test.drop(['STDNT_TEST_ENTRANCE_COMB','DISTANCE_FROM_HOME','absentee_hrs_s'], axis=1)
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train_final,Y_train)
y_pred=model.predict(X_test_final)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test,predictions))
ncols_new=X_test_final.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8205882352941176
('HIGH_SCHL_GPA', 0.047219235)
('FIRST_TERM_ATTEMPT_HRS', 0.038052246)
('FIRST_TERM_EARNED_HRS', 0.045758158)
('SECOND_TERM_ATTEMPT_HRS', 0.37348396)
('SECOND_TERM_EARNED_HRS', 0.103096254)
('EST_FAM_CONTRIBUTION', 0.048897576)
('absentee_hrs_f', 0.045586586)
('intl_flag', 0.058613952)
('grd_grp_Good', 0.17962703)
('grd_grp_Poor', 0.05966505)


In [70]:
X_final= std_data1[['SECOND_TERM_ATTEMPT_HRS','SECOND_TERM_EARNED_HRS','grd_grp_Good','grd_grp_Poor']]
Y_final = std_data1['ret_2nd_yr']

from sklearn.model_selection import train_test_split
X_train_final,X_test_final,Y_train_final,Y_test_final = train_test_split(X_final,Y_final,test_size = 0.30,random_state=200)

In [71]:
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train_final,Y_train_final)
y_pred=model.predict(X_test_final)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test_final,predictions))
ncols_new=X_test_final.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8245098039215686
('SECOND_TERM_ATTEMPT_HRS', 0.41186368)
('SECOND_TERM_EARNED_HRS', 0.12041624)
('grd_grp_Good', 0.40936506)
('grd_grp_Poor', 0.058355026)


In [72]:
X_train_final=X_train_final[['SECOND_TERM_ATTEMPT_HRS','grd_grp_Good']]
X_test_final=X_test_final[['SECOND_TERM_ATTEMPT_HRS','grd_grp_Good']]
model = XGBClassifier(gamma=0,learning_rate=0.05)
model.fit(X_train_final,Y_train_final)
y_pred=model.predict(X_test_final)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test_final,predictions))
ncols_new=X_test_final.columns
for feature  in zip(ncols_new, model.feature_importances_):
    print(feature)

0.8254901960784313
('SECOND_TERM_ATTEMPT_HRS', 0.55469674)
('grd_grp_Good', 0.4453033)


In [73]:
grid=GridSearchCV(xgclf,parameters,scoring="accuracy",cv=5)
grid.fit(X_train_final,Y_train_final)
y_pred=grid.predict(X_test_final)
predictions=[round(value) for value in y_pred]
print(accuracy_score(Y_test_final,predictions))

0.8254901960784313


In [74]:
grid.best_params_

{'gamma': 0.2, 'learning_rate': 0.35}

In [75]:
grid.best_estimator_

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.35, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [76]:
for estimator in range(100,1000,100):
    clf=RandomForestClassifier(n_estimators=estimator,oob_score=True,n_jobs=-1,random_state=200)
    clf.fit(X_train_final,Y_train_final)
    oob=clf.oob_score_
    print('For n_estimators='+str(estimator))
    print('For OOB score is '+str(oob))
    print('****************************')
    

For n_estimators=100
For OOB score is 0.834873949579832
****************************
For n_estimators=200
For OOB score is 0.8365546218487395
****************************
For n_estimators=300
For OOB score is 0.8352941176470589
****************************
For n_estimators=400
For OOB score is 0.8357142857142857
****************************
For n_estimators=500
For OOB score is 0.834873949579832
****************************
For n_estimators=600
For OOB score is 0.8340336134453782
****************************
For n_estimators=700
For OOB score is 0.8340336134453782
****************************
For n_estimators=800
For OOB score is 0.8340336134453782
****************************
For n_estimators=900
For OOB score is 0.8344537815126051
****************************


In [78]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train_final,Y_train_final)
y_pred_logreg=logreg.predict(X_test_final)
print(logreg.score(X_test_final,Y_test_final))

0.8245098039215686


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [79]:
confusion_matrix_logreg=confusion_matrix(Y_test_final,y_pred_logreg)
print(confusion_matrix_logreg)

[[788  13]
 [166  53]]


In [82]:
clf=RandomForestClassifier(n_estimators=200,oob_score=True,n_jobs=-1,random_state=200)
clf.fit(X_train_final,Y_train_final)
oob=clf.oob_score_
print('For OOB score is '+str(oob))
print('****************************')
    

For OOB score is 0.8365546218487395
****************************


In [83]:
clf.feature_importances_

array([0.84786848, 0.15213152])